In [21]:
WITH prices_raw AS (
    SELECT
        ep.*,
        COUNT(*) FILTER (
            WHERE ep.ResolutionCode = 'PT60M'
        ) OVER (
            PARTITION BY ep.AreaCode, ep."DateTime(UTC)"
        ) AS cnt_60m_same_ts
    FROM read_csv_auto(
            '2025_01_EnergyPrices_12.1.D_r3.csv',
            strict_mode=false,
            null_padding=true
        ) AS ep
    WHERE
        ep.ContractType = 'Day-ahead'
        AND ep.Sequence NOT IN ('2', '3')
),

prices_dedup AS (
    -- - if PT60M & PT15M exist → only PT60M
    -- - if only PT15M  → PT15M
    -- - if only PT60M  → PT60M
    SELECT *
    FROM prices_raw
    WHERE
        ResolutionCode = 'PT60M'
        OR (ResolutionCode = 'PT15M' AND cnt_60m_same_ts = 0)
),

-- 1. cumulative negative pricing hours from prices_dedup
neg_hours AS (
    SELECT
        AreaDisplayName AS country,
        SUM(
            CASE
                WHEN ResolutionCode = 'PT60M' AND "Price[Currency/MWh]" < 0 THEN 1.0
                WHEN ResolutionCode = 'PT15M' AND "Price[Currency/MWh]" < 0 THEN 0.25
                ELSE 0.0
            END
        ) AS neg_hours_total
    FROM prices_dedup
    GROUP BY AreaDisplayName
),

-- 2. Avg. market price )
avg_price AS (
    SELECT
        AreaDisplayName AS country,
        AVG("Price[Currency/MWh]") AS avg_price_eur_per_mwh
    FROM prices_dedup
    GROUP BY AreaDisplayName
),

-- 3. joining on time and area codes 
joined AS (
    SELECT
        ep.AreaDisplayName AS country,
        ep.AreaCode,
        ep."DateTime(UTC)",
        ep.ResolutionCode,

        -- set resolutions codes
        CASE
            WHEN ep.ResolutionCode = 'PT15M' THEN 0.25
            WHEN ep.ResolutionCode = 'PT60M' THEN 1.0
            ELSE 1.0
        END AS interval_hours,

        ep."Price[Currency/MWh]" AS price_raw,

        ag."ActualGenerationOutput" AS gen_mw
    FROM prices_dedup AS ep
    JOIN read_csv_auto(
            '2025_01_AggregatedGenerationPerType_16.1.B_C.csv',
            strict_mode=false,
            null_padding=true
        ) AS ag
      ON ep.AreaCode        = ag.AreaCode
     AND ep."DateTime(UTC)" = ag."DateTime"
    WHERE
        ag.ProductionType = 'Solar'
        AND ag."ActualGenerationOutput" > 0
),

-- 4. Solar generation during negative pricing hours (in MWh)
solar_energy AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours) AS solar_mwh_total,
        SUM(
            CASE
                WHEN price_raw < 0 THEN gen_mw * interval_hours
                ELSE 0
            END
        ) AS solar_mwh_at_neg_price
    FROM joined
    GROUP BY country
),

-- 5. Capture Price (€/MWh)
capture AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours * price_raw)
        / NULLIF(SUM(gen_mw * interval_hours), 0) AS capture_eur_per_mwh
    FROM joined
    GROUP BY country
),

-- 6. Capture Price with floor price of 0 (€/MWh)
capture_floor0 AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours * CASE WHEN price_raw < 0 THEN 0 ELSE price_raw END)
        / NULLIF(SUM(gen_mw * interval_hours), 0) AS capture_price_floor0_eur_per_mwh
    FROM joined
    GROUP BY country
)

SELECT
    a.country,
    n.neg_hours_total,

    -- avg market price electrcity (Day-ahead) in ct/kWh
    ROUND(a.avg_price_eur_per_mwh * 0.1, 3) AS avg_price_ct_per_kwh,

    -- Capture Price in ct/kWh
    ROUND(c.capture_eur_per_mwh * 0.1, 3)   AS capture_price_ct_per_kwh,
	
    -- Capture Price with floor price of 0 in ct/kWh
    ROUND(cf.capture_price_floor0_eur_per_mwh * 0.1, 3) AS capture_price_floor0_ct_kwh,

    -- Capture Rate in % 
    ROUND(
        100.0 * c.capture_eur_per_mwh
        / NULLIF(a.avg_price_eur_per_mwh, 0),
        2
    ) AS capture_rate_percent,

    -- volume share of solar power generated during negative pricing hours (in %)
    ROUND(
        100.0 * s.solar_mwh_at_neg_price
        / NULLIF(s.solar_mwh_total, 0),
        2
    ) AS share_solar_at_neg_price_percent

FROM avg_price a
LEFT JOIN neg_hours    n ON a.country = n.country
LEFT JOIN capture      c ON a.country = c.country
LEFT JOIN solar_energy s ON a.country = s.country
LEFT JOIN capture_floor0 cf ON a.country = cf.country
ORDER BY a.country;

,country,neg_hours_total,avg_price_ct_per_kwh,capture_price_ct_per_kwh,capture_price_floor0_ct_kwh,capture_rate_percent,share_solar_at_neg_price_percent
0,Austria (AT),0.0,13.389,13.018,13.018,97.23,0.00
1,Belgium (BE),0.0,11.217,11.701,11.701,104.31,0.00
2,Bulgaria (BG),0.0,13.853,12.480,12.480,90.09,0.00
3,Croatia (HR),0.0,13.600,12.680,12.680,93.24,0.00
4,Czech Republic (CZ),0.0,12.855,12.565,12.565,97.75,0.00
5,DE-LU,14.0,11.432,11.650,11.651,101.91,4.95
6,DK1,0.0,9.815,11.599,11.599,118.18,0.00
7,DK2,0.0,10.030,11.854,11.854,118.18,0.00
8,Estonia (EE),0.0,9.219,10.866,10.866,117.86,0.00
9,Finland (FI),0.0,5.282,6.765,6.765,128.07,0.00


In [20]:
WITH prices_raw AS (
    SELECT
        ep.*,
        COUNT(*) FILTER (
            WHERE ep.ResolutionCode = 'PT60M'
        ) OVER (
            PARTITION BY ep.AreaCode, ep."DateTime(UTC)"
        ) AS cnt_60m_same_ts
    FROM read_csv_auto(
            '2025_11_EnergyPrices_12.1.D_r3.csv',
            strict_mode=false,
            null_padding=true
        ) AS ep
    WHERE
        ep.ContractType = 'Day-ahead'
        AND ep.Sequence NOT IN ('2', '3')
),

prices_dedup AS (
    -- - if PT60M & PT15M exist → only PT60M
    -- - if only PT15M  → PT15M
    -- - if only PT60M  → PT60M
    SELECT *
    FROM prices_raw
    WHERE
        ResolutionCode = 'PT60M'
        OR (ResolutionCode = 'PT15M' AND cnt_60m_same_ts = 0)
),

-- 1. cumulative negative pricing hours from prices_dedup
neg_hours AS (
    SELECT
        AreaDisplayName AS country,
        SUM(
            CASE
                WHEN ResolutionCode = 'PT60M' AND "Price[Currency/MWh]" < 0 THEN 1.0
                WHEN ResolutionCode = 'PT15M' AND "Price[Currency/MWh]" < 0 THEN 0.25
                ELSE 0.0
            END
        ) AS neg_hours_total
    FROM prices_dedup
    GROUP BY AreaDisplayName
),

-- 2. Avg. market price )
avg_price AS (
    SELECT
        AreaDisplayName AS country,
        AVG("Price[Currency/MWh]") AS avg_price_eur_per_mwh
    FROM prices_dedup
    GROUP BY AreaDisplayName
),

-- 3. joining on time and area codes 
joined AS (
    SELECT
        ep.AreaDisplayName AS country,
        ep.AreaCode,
        ep."DateTime(UTC)",
        ep.ResolutionCode,

        -- set resolutions codes
        CASE
            WHEN ep.ResolutionCode = 'PT15M' THEN 0.25
            WHEN ep.ResolutionCode = 'PT60M' THEN 1.0
            ELSE 1.0
        END AS interval_hours,

        ep."Price[Currency/MWh]" AS price_raw,

        ag."ActualGenerationOutput" AS gen_mw
    FROM prices_dedup AS ep
    JOIN read_csv_auto(
            '2025_02_AggregatedGenerationPerType_16.1.B_C.csv',
            strict_mode=false,
            null_padding=true
        ) AS ag
      ON ep.AreaCode        = ag.AreaCode
     AND ep."DateTime(UTC)" = ag."DateTime"
    WHERE
        ag.ProductionType = 'Solar'
        AND ag."ActualGenerationOutput" > 0
),

-- 4. Solar generation during negative pricing hours (in MWh)
solar_energy AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours) AS solar_mwh_total,
        SUM(
            CASE
                WHEN price_raw < 0 THEN gen_mw * interval_hours
                ELSE 0
            END
        ) AS solar_mwh_at_neg_price
    FROM joined
    GROUP BY country
),

-- 5. Capture Price (€/MWh)
capture AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours * price_raw)
        / NULLIF(SUM(gen_mw * interval_hours), 0) AS capture_eur_per_mwh
    FROM joined
    GROUP BY country
),

-- 6. Capture Price with floor price of 0 (€/MWh)
capture_floor0 AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours * CASE WHEN price_raw < 0 THEN 0 ELSE price_raw END)
        / NULLIF(SUM(gen_mw * interval_hours), 0) AS capture_price_floor0_eur_per_mwh
    FROM joined
    GROUP BY country
)

SELECT
    a.country,
    n.neg_hours_total,

    -- avg market price electrcity (Day-ahead) in ct/kWh
    ROUND(a.avg_price_eur_per_mwh * 0.1, 3) AS avg_price_ct_per_kwh,

    -- Capture Price in ct/kWh
    ROUND(c.capture_eur_per_mwh * 0.1, 3)   AS capture_price_ct_per_kwh,
	
    -- Capture Price with floor price of 0 in ct/kWh
    ROUND(cf.capture_price_floor0_eur_per_mwh * 0.1, 3) AS capture_price_floor0_ct_kwh,

    -- Capture Rate in % 
    ROUND(
        100.0 * c.capture_eur_per_mwh
        / NULLIF(a.avg_price_eur_per_mwh, 0),
        2
    ) AS capture_rate_percent,

    -- volume share of solar power generated during negative pricing hours (in %)
    ROUND(
        100.0 * s.solar_mwh_at_neg_price
        / NULLIF(s.solar_mwh_total, 0),
        2
    ) AS share_solar_at_neg_price_percent

FROM avg_price a
LEFT JOIN neg_hours    n ON a.country = n.country
LEFT JOIN capture      c ON a.country = c.country
LEFT JOIN solar_energy s ON a.country = s.country
LEFT JOIN capture_floor0 cf ON a.country = cf.country
ORDER BY a.country;

Error: Conversion Error: CSV Error on Line: 1389672
Original Line: 
2025-02-
Error when converting column "DateTime". Could not convert string "2025-02-" to 'TIMESTAMP'

Column DateTime is being converted as type TIMESTAMP
This type was auto-detected from the CSV file.
Possible solutions:
* Override the type for this column manually by setting the type explicitly, e.g. types={'DateTime': 'VARCHAR'}
* Set the sample size to a larger value to enable the auto-detection to scan more values, e.g. sample_size=-1
* Use a COPY statement to automatically derive types from an existing table.
* Check whether the null string value is set correctly (e.g., nullstr = 'N/A')

  file = 2025_02_AggregatedGenerationPerType_16.1.B_C.csv
  delimiter = 	 (Auto-Detected)
  quote = \0 (Auto-Detected)
  escape = \0 (Auto-Detected)
  new_line = \r\n (Auto-Detected)
  header = true (Auto-Detected)
  skip_rows = 0 (Auto-Detected)
  comment = \0 (Auto-Detected)
  strict_mode = false (Set By User)
  date_format =  

In [ ]:
WITH prices_raw AS (
    SELECT
        ep.*,
        COUNT(*) FILTER (
            WHERE ep.ResolutionCode = 'PT60M'
        ) OVER (
            PARTITION BY ep.AreaCode, ep."DateTime(UTC)"
        ) AS cnt_60m_same_ts
    FROM read_csv_auto(
            '2025_03_EnergyPrices_12.1.D_r3.csv',
            strict_mode=false,
            null_padding=true
        ) AS ep
    WHERE
        ep.ContractType = 'Day-ahead'
        AND ep.Sequence NOT IN ('2', '3')
),

prices_dedup AS (
    -- - if PT60M & PT15M exist → only PT60M
    -- - if only PT15M  → PT15M
    -- - if only PT60M  → PT60M
    SELECT *
    FROM prices_raw
    WHERE
        ResolutionCode = 'PT60M'
        OR (ResolutionCode = 'PT15M' AND cnt_60m_same_ts = 0)
),

-- 1. cumulative negative pricing hours from prices_dedup
neg_hours AS (
    SELECT
        AreaDisplayName AS country,
        SUM(
            CASE
                WHEN ResolutionCode = 'PT60M' AND "Price[Currency/MWh]" < 0 THEN 1.0
                WHEN ResolutionCode = 'PT15M' AND "Price[Currency/MWh]" < 0 THEN 0.25
                ELSE 0.0
            END
        ) AS neg_hours_total
    FROM prices_dedup
    GROUP BY AreaDisplayName
),

-- 2. Avg. market price )
avg_price AS (
    SELECT
        AreaDisplayName AS country,
        AVG("Price[Currency/MWh]") AS avg_price_eur_per_mwh
    FROM prices_dedup
    GROUP BY AreaDisplayName
),

-- 3. joining on time and area codes 
joined AS (
    SELECT
        ep.AreaDisplayName AS country,
        ep.AreaCode,
        ep."DateTime(UTC)",
        ep.ResolutionCode,

        -- set resolutions codes
        CASE
            WHEN ep.ResolutionCode = 'PT15M' THEN 0.25
            WHEN ep.ResolutionCode = 'PT60M' THEN 1.0
            ELSE 1.0
        END AS interval_hours,

        ep."Price[Currency/MWh]" AS price_raw,

        ag."ActualGenerationOutput" AS gen_mw
    FROM prices_dedup AS ep
    JOIN read_csv_auto(
            '2025_03_AggregatedGenerationPerType_16.1.B_C.csv',
            strict_mode=false,
            null_padding=true
        ) AS ag
      ON ep.AreaCode        = ag.AreaCode
     AND ep."DateTime(UTC)" = ag."DateTime"
    WHERE
        ag.ProductionType = 'Solar'
        AND ag."ActualGenerationOutput" > 0
),

-- 4. Solar generation during negative pricing hours (in MWh)
solar_energy AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours) AS solar_mwh_total,
        SUM(
            CASE
                WHEN price_raw < 0 THEN gen_mw * interval_hours
                ELSE 0
            END
        ) AS solar_mwh_at_neg_price
    FROM joined
    GROUP BY country
),

-- 5. Capture Price (€/MWh)
capture AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours * price_raw)
        / NULLIF(SUM(gen_mw * interval_hours), 0) AS capture_eur_per_mwh
    FROM joined
    GROUP BY country
),

-- 6. Capture Price with floor price of 0 (€/MWh)
capture_floor0 AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours * CASE WHEN price_raw < 0 THEN 0 ELSE price_raw END)
        / NULLIF(SUM(gen_mw * interval_hours), 0) AS capture_price_floor0_eur_per_mwh
    FROM joined
    GROUP BY country
)

SELECT
    a.country,
    n.neg_hours_total,

    -- avg market price electrcity (Day-ahead) in ct/kWh
    ROUND(a.avg_price_eur_per_mwh * 0.1, 3) AS avg_price_ct_per_kwh,

    -- Capture Price in ct/kWh
    ROUND(c.capture_eur_per_mwh * 0.1, 3)   AS capture_price_ct_per_kwh,
	
    -- Capture Price with floor price of 0 in ct/kWh
    ROUND(cf.capture_price_floor0_eur_per_mwh * 0.1, 3) AS capture_price_floor0_ct_kwh,

    -- Capture Rate in % 
    ROUND(
        100.0 * c.capture_eur_per_mwh
        / NULLIF(a.avg_price_eur_per_mwh, 0),
        2
    ) AS capture_rate_percent,

    -- volume share of solar power generated during negative pricing hours (in %)
    ROUND(
        100.0 * s.solar_mwh_at_neg_price
        / NULLIF(s.solar_mwh_total, 0),
        2
    ) AS share_solar_at_neg_price_percent

FROM avg_price a
LEFT JOIN neg_hours    n ON a.country = n.country
LEFT JOIN capture      c ON a.country = c.country
LEFT JOIN solar_energy s ON a.country = s.country
LEFT JOIN capture_floor0 cf ON a.country = cf.country
ORDER BY a.country;

Error: Conversion Error: CSV Error on Line: 1389193
Original Line: 
2025-03-
Error when converting column "DateTime". Could not convert string "2025-03-" to 'TIMESTAMP'

Column DateTime is being converted as type TIMESTAMP
This type was auto-detected from the CSV file.
Possible solutions:
* Override the type for this column manually by setting the type explicitly, e.g. types={'DateTime': 'VARCHAR'}
* Set the sample size to a larger value to enable the auto-detection to scan more values, e.g. sample_size=-1
* Use a COPY statement to automatically derive types from an existing table.
* Check whether the null string value is set correctly (e.g., nullstr = 'N/A')

  file = 2025_03_AggregatedGenerationPerType_16.1.B_C.csv
  delimiter = 	 (Auto-Detected)
  quote = \0 (Auto-Detected)
  escape = \0 (Auto-Detected)
  new_line = \r\n (Auto-Detected)
  header = true (Auto-Detected)
  skip_rows = 0 (Auto-Detected)
  comment = \0 (Auto-Detected)
  strict_mode = false (Set By User)
  date_format =  

In [22]:
WITH prices_raw AS (
    SELECT
        ep.*,
        COUNT(*) FILTER (
            WHERE ep.ResolutionCode = 'PT60M'
        ) OVER (
            PARTITION BY ep.AreaCode, ep."DateTime(UTC)"
        ) AS cnt_60m_same_ts
    FROM read_csv_auto(
            '2025_04_EnergyPrices_12.1.D_r3.csv',
            strict_mode=false,
            null_padding=true
        ) AS ep
    WHERE
        ep.ContractType = 'Day-ahead'
        AND ep.Sequence NOT IN ('2', '3')
),

prices_dedup AS (
    -- - if PT60M & PT15M exist → only PT60M
    -- - if only PT15M  → PT15M
    -- - if only PT60M  → PT60M
    SELECT *
    FROM prices_raw
    WHERE
        ResolutionCode = 'PT60M'
        OR (ResolutionCode = 'PT15M' AND cnt_60m_same_ts = 0)
),

-- 1. cumulative negative pricing hours from prices_dedup
neg_hours AS (
    SELECT
        AreaDisplayName AS country,
        SUM(
            CASE
                WHEN ResolutionCode = 'PT60M' AND "Price[Currency/MWh]" < 0 THEN 1.0
                WHEN ResolutionCode = 'PT15M' AND "Price[Currency/MWh]" < 0 THEN 0.25
                ELSE 0.0
            END
        ) AS neg_hours_total
    FROM prices_dedup
    GROUP BY AreaDisplayName
),

-- 2. Avg. market price )
avg_price AS (
    SELECT
        AreaDisplayName AS country,
        AVG("Price[Currency/MWh]") AS avg_price_eur_per_mwh
    FROM prices_dedup
    GROUP BY AreaDisplayName
),

-- 3. joining on time and area codes 
joined AS (
    SELECT
        ep.AreaDisplayName AS country,
        ep.AreaCode,
        ep."DateTime(UTC)",
        ep.ResolutionCode,

        -- set resolutions codes
        CASE
            WHEN ep.ResolutionCode = 'PT15M' THEN 0.25
            WHEN ep.ResolutionCode = 'PT60M' THEN 1.0
            ELSE 1.0
        END AS interval_hours,

        ep."Price[Currency/MWh]" AS price_raw,

        ag."ActualGenerationOutput" AS gen_mw
    FROM prices_dedup AS ep
    JOIN read_csv_auto(
            '2025_04_AggregatedGenerationPerType_16.1.B_C.csv',
            strict_mode=false,
            null_padding=true
        ) AS ag
      ON ep.AreaCode        = ag.AreaCode
     AND ep."DateTime(UTC)" = ag."DateTime"
    WHERE
        ag.ProductionType = 'Solar'
        AND ag."ActualGenerationOutput" > 0
),

-- 4. Solar generation during negative pricing hours (in MWh)
solar_energy AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours) AS solar_mwh_total,
        SUM(
            CASE
                WHEN price_raw < 0 THEN gen_mw * interval_hours
                ELSE 0
            END
        ) AS solar_mwh_at_neg_price
    FROM joined
    GROUP BY country
),

-- 5. Capture Price (€/MWh)
capture AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours * price_raw)
        / NULLIF(SUM(gen_mw * interval_hours), 0) AS capture_eur_per_mwh
    FROM joined
    GROUP BY country
),

-- 6. Capture Price with floor price of 0 (€/MWh)
capture_floor0 AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours * CASE WHEN price_raw < 0 THEN 0 ELSE price_raw END)
        / NULLIF(SUM(gen_mw * interval_hours), 0) AS capture_price_floor0_eur_per_mwh
    FROM joined
    GROUP BY country
)

SELECT
    a.country,
    n.neg_hours_total,

    -- avg market price electrcity (Day-ahead) in ct/kWh
    ROUND(a.avg_price_eur_per_mwh * 0.1, 3) AS avg_price_ct_per_kwh,

    -- Capture Price in ct/kWh
    ROUND(c.capture_eur_per_mwh * 0.1, 3)   AS capture_price_ct_per_kwh,
	
    -- Capture Price with floor price of 0 in ct/kWh
    ROUND(cf.capture_price_floor0_eur_per_mwh * 0.1, 3) AS capture_price_floor0_ct_kwh,

    -- Capture Rate in % 
    ROUND(
        100.0 * c.capture_eur_per_mwh
        / NULLIF(a.avg_price_eur_per_mwh, 0),
        2
    ) AS capture_rate_percent,

    -- volume share of solar power generated during negative pricing hours (in %)
    ROUND(
        100.0 * s.solar_mwh_at_neg_price
        / NULLIF(s.solar_mwh_total, 0),
        2
    ) AS share_solar_at_neg_price_percent

FROM avg_price a
LEFT JOIN neg_hours    n ON a.country = n.country
LEFT JOIN capture      c ON a.country = c.country
LEFT JOIN solar_energy s ON a.country = s.country
LEFT JOIN capture_floor0 cf ON a.country = cf.country
ORDER BY a.country;

,country,neg_hours_total,avg_price_ct_per_kwh,capture_price_ct_per_kwh,capture_price_floor0_ct_kwh,capture_rate_percent,share_solar_at_neg_price_percent
0,Austria (AT),76.0,8.104,2.801,3.764,34.56,31.39
1,Belgium (BE),80.0,7.336,2.438,3.304,33.23,34.18
2,Bulgaria (BG),46.0,8.494,5.488,5.684,64.60,9.45
3,Croatia (HR),68.0,8.478,5.257,5.544,62.01,13.43
4,Czech Republic (CZ),72.0,7.964,3.494,4.121,43.87,26.64
5,DE-LU,75.0,7.793,3.009,3.677,38.62,31.76
6,DK1,74.0,7.515,4.071,4.184,54.17,20.66
7,DK2,58.0,7.580,4.001,4.085,52.78,21.00
8,Estonia (EE),35.0,7.291,3.033,3.051,41.61,14.43
9,Finland (FI),69.0,4.782,2.812,2.832,58.81,19.12


In [ ]:
WITH prices_raw AS (
    SELECT
        ep.*,
        COUNT(*) FILTER (
            WHERE ep.ResolutionCode = 'PT60M'
        ) OVER (
            PARTITION BY ep.AreaCode, ep."DateTime(UTC)"
        ) AS cnt_60m_same_ts
    FROM read_csv_auto(
            '2025_05_EnergyPrices_12.1.D_r3.csv',
            strict_mode=false,
            null_padding=true,
            columns = {
                "DateTime(UTC)": "VARCHAR"
            }
        ) AS ep
    WHERE
        ep.ContractType = 'Day-ahead'
        AND ep.Sequence NOT IN ('2', '3')
),

prices_dedup AS (
    -- - if PT60M & PT15M exist → only PT60M
    -- - if only PT15M  → PT15M
    -- - if only PT60M  → PT60M
    SELECT *
    FROM prices_raw
    WHERE
        ResolutionCode = 'PT60M'
        OR (ResolutionCode = 'PT15M' AND cnt_60m_same_ts = 0)
),

-- 1. cumulative negative pricing hours from prices_dedup
neg_hours AS (
    SELECT
        AreaDisplayName AS country,
        SUM(
            CASE
                WHEN ResolutionCode = 'PT60M' AND "Price[Currency/MWh]" < 0 THEN 1.0
                WHEN ResolutionCode = 'PT15M' AND "Price[Currency/MWh]" < 0 THEN 0.25
                ELSE 0.0
            END
        ) AS neg_hours_total
    FROM prices_dedup
    GROUP BY AreaDisplayName
),

-- 2. Avg. market price )
avg_price AS (
    SELECT
        AreaDisplayName AS country,
        AVG("Price[Currency/MWh]") AS avg_price_eur_per_mwh
    FROM prices_dedup
    GROUP BY AreaDisplayName
),

-- 3. joining on time and area codes 
joined AS (
    SELECT
        ep.AreaDisplayName AS country,
        ep.AreaCode,
        ep."DateTime(UTC)",
        ep.ResolutionCode,

        -- set resolutions codes
        CASE
            WHEN ep.ResolutionCode = 'PT15M' THEN 0.25
            WHEN ep.ResolutionCode = 'PT60M' THEN 1.0
            ELSE 1.0
        END AS interval_hours,

        ep."Price[Currency/MWh]" AS price_raw,

        ag."ActualGenerationOutput" AS gen_mw
    FROM prices_dedup AS ep
    JOIN read_csv_auto(
            '2025_05_AggregatedGenerationPerType_16.1.B_C.csv',
            strict_mode=false,
            null_padding=true,
            columns = {
                "DateTime": "VARCHAR"
            }
        ) AS ag
      ON ep.AreaCode        = ag.AreaCode
     AND ep."DateTime(UTC)" = ag."DateTime"
    WHERE
        ag.ProductionType = 'Solar'
        AND ag."ActualGenerationOutput" > 0
),

-- 4. Solar generation during negative pricing hours (in MWh)
solar_energy AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours) AS solar_mwh_total,
        SUM(
            CASE
                WHEN price_raw < 0 THEN gen_mw * interval_hours
                ELSE 0
            END
        ) AS solar_mwh_at_neg_price
    FROM joined
    GROUP BY country
),

-- 5. Capture Price (€/MWh)
capture AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours * price_raw)
        / NULLIF(SUM(gen_mw * interval_hours), 0) AS capture_eur_per_mwh
    FROM joined
    GROUP BY country
),

-- 6. Capture Price with floor price of 0 (€/MWh)
capture_floor0 AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours * CASE WHEN price_raw < 0 THEN 0 ELSE price_raw END)
        / NULLIF(SUM(gen_mw * interval_hours), 0) AS capture_price_floor0_eur_per_mwh
    FROM joined
    GROUP BY country
)

SELECT
    a.country,
    n.neg_hours_total,

    -- avg market price electrcity (Day-ahead) in ct/kWh
    ROUND(a.avg_price_eur_per_mwh * 0.1, 3) AS avg_price_ct_per_kwh,

    -- Capture Price in ct/kWh
    ROUND(c.capture_eur_per_mwh * 0.1, 3)   AS capture_price_ct_per_kwh,
	
    -- Capture Price with floor price of 0 in ct/kWh
    ROUND(cf.capture_price_floor0_eur_per_mwh * 0.1, 3) AS capture_price_floor0_ct_kwh,

    -- Capture Rate in % 
    ROUND(
        100.0 * c.capture_eur_per_mwh
        / NULLIF(a.avg_price_eur_per_mwh, 0),
        2
    ) AS capture_rate_percent,

    -- volume share of solar power generated during negative pricing hours (in %)
    ROUND(
        100.0 * s.solar_mwh_at_neg_price
        / NULLIF(s.solar_mwh_total, 0),
        2
    ) AS share_solar_at_neg_price_percent

FROM avg_price a
LEFT JOIN neg_hours    n ON a.country = n.country
LEFT JOIN capture      c ON a.country = c.country
LEFT JOIN solar_energy s ON a.country = s.country
LEFT JOIN capture_floor0 cf ON a.country = cf.country
ORDER BY a.country;

Error: Binder Error: Table "ep" does not have a column named "ContractType"

Candidate bindings: : "DateTime(UTC)"

LINE 18:         ep.ContractType = 'Day-ahead'
                 ^

In [23]:
WITH prices_raw AS (
    SELECT
        ep.*,
        COUNT(*) FILTER (
            WHERE ep.ResolutionCode = 'PT60M'
        ) OVER (
            PARTITION BY ep.AreaCode, ep."DateTime(UTC)"
        ) AS cnt_60m_same_ts
    FROM read_csv_auto(
            '2025_06_EnergyPrices_12.1.D_r3.csv',
            strict_mode=false,
            null_padding=true
        ) AS ep
    WHERE
        ep.ContractType = 'Day-ahead'
        AND ep.Sequence NOT IN ('2', '3')
),

prices_dedup AS (
    -- - if PT60M & PT15M exist → only PT60M
    -- - if only PT15M  → PT15M
    -- - if only PT60M  → PT60M
    SELECT *
    FROM prices_raw
    WHERE
        ResolutionCode = 'PT60M'
        OR (ResolutionCode = 'PT15M' AND cnt_60m_same_ts = 0)
),

-- 1. cumulative negative pricing hours from prices_dedup
neg_hours AS (
    SELECT
        AreaDisplayName AS country,
        SUM(
            CASE
                WHEN ResolutionCode = 'PT60M' AND "Price[Currency/MWh]" < 0 THEN 1.0
                WHEN ResolutionCode = 'PT15M' AND "Price[Currency/MWh]" < 0 THEN 0.25
                ELSE 0.0
            END
        ) AS neg_hours_total
    FROM prices_dedup
    GROUP BY AreaDisplayName
),

-- 2. Avg. market price )
avg_price AS (
    SELECT
        AreaDisplayName AS country,
        AVG("Price[Currency/MWh]") AS avg_price_eur_per_mwh
    FROM prices_dedup
    GROUP BY AreaDisplayName
),

-- 3. joining on time and area codes 
joined AS (
    SELECT
        ep.AreaDisplayName AS country,
        ep.AreaCode,
        ep."DateTime(UTC)",
        ep.ResolutionCode,

        -- set resolutions codes
        CASE
            WHEN ep.ResolutionCode = 'PT15M' THEN 0.25
            WHEN ep.ResolutionCode = 'PT60M' THEN 1.0
            ELSE 1.0
        END AS interval_hours,

        ep."Price[Currency/MWh]" AS price_raw,

        ag."ActualGenerationOutput" AS gen_mw
    FROM prices_dedup AS ep
    JOIN read_csv_auto(
            '2025_06_AggregatedGenerationPerType_16.1.B_C.csv',
            strict_mode=false,
            null_padding=true
        ) AS ag
      ON ep.AreaCode        = ag.AreaCode
     AND ep."DateTime(UTC)" = ag."DateTime"
    WHERE
        ag.ProductionType = 'Solar'
        AND ag."ActualGenerationOutput" > 0
),

-- 4. Solar generation during negative pricing hours (in MWh)
solar_energy AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours) AS solar_mwh_total,
        SUM(
            CASE
                WHEN price_raw < 0 THEN gen_mw * interval_hours
                ELSE 0
            END
        ) AS solar_mwh_at_neg_price
    FROM joined
    GROUP BY country
),

-- 5. Capture Price (€/MWh)
capture AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours * price_raw)
        / NULLIF(SUM(gen_mw * interval_hours), 0) AS capture_eur_per_mwh
    FROM joined
    GROUP BY country
),

-- 6. Capture Price with floor price of 0 (€/MWh)
capture_floor0 AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours * CASE WHEN price_raw < 0 THEN 0 ELSE price_raw END)
        / NULLIF(SUM(gen_mw * interval_hours), 0) AS capture_price_floor0_eur_per_mwh
    FROM joined
    GROUP BY country
)

SELECT
    a.country,
    n.neg_hours_total,

    -- avg market price electrcity (Day-ahead) in ct/kWh
    ROUND(a.avg_price_eur_per_mwh * 0.1, 3) AS avg_price_ct_per_kwh,

    -- Capture Price in ct/kWh
    ROUND(c.capture_eur_per_mwh * 0.1, 3)   AS capture_price_ct_per_kwh,
	
    -- Capture Price with floor price of 0 in ct/kWh
    ROUND(cf.capture_price_floor0_eur_per_mwh * 0.1, 3) AS capture_price_floor0_ct_kwh,

    -- Capture Rate in % 
    ROUND(
        100.0 * c.capture_eur_per_mwh
        / NULLIF(a.avg_price_eur_per_mwh, 0),
        2
    ) AS capture_rate_percent,

    -- volume share of solar power generated during negative pricing hours (in %)
    ROUND(
        100.0 * s.solar_mwh_at_neg_price
        / NULLIF(s.solar_mwh_total, 0),
        2
    ) AS share_solar_at_neg_price_percent

FROM avg_price a
LEFT JOIN neg_hours    n ON a.country = n.country
LEFT JOIN capture      c ON a.country = c.country
LEFT JOIN solar_energy s ON a.country = s.country
LEFT JOIN capture_floor0 cf ON a.country = cf.country
ORDER BY a.country;

,country,neg_hours_total,avg_price_ct_per_kwh,capture_price_ct_per_kwh,capture_price_floor0_ct_kwh,capture_rate_percent,share_solar_at_neg_price_percent
0,Austria (AT),115.0,6.650,1.435,1.868,21.57,48.40
1,Belgium (BE),127.0,6.536,2.123,2.448,32.48,47.00
2,Bulgaria (BG),28.0,8.531,4.536,4.555,53.16,5.77
3,Croatia (HR),48.0,8.605,4.071,4.142,47.31,9.29
4,Czech Republic (CZ),69.0,7.710,2.931,3.137,38.02,25.62
5,DE-LU,141.0,6.403,2.014,2.391,31.46,49.17
6,DK1,116.0,5.882,2.967,3.034,50.44,27.75
7,DK2,66.0,5.649,2.310,2.381,40.89,20.24
8,Estonia (EE),70.0,4.114,3.154,3.211,76.67,12.21
9,Finland (FI),126.0,1.858,2.759,2.818,148.47,13.16


In [24]:
WITH prices_raw AS (
    SELECT
        ep.*,
        COUNT(*) FILTER (
            WHERE ep.ResolutionCode = 'PT60M'
        ) OVER (
            PARTITION BY ep.AreaCode, ep."DateTime(UTC)"
        ) AS cnt_60m_same_ts
    FROM read_csv_auto(
            '2025_07_EnergyPrices_12.1.D_r3.csv',
            strict_mode=false,
            null_padding=true
        ) AS ep
    WHERE
        ep.ContractType = 'Day-ahead'
        AND ep.Sequence NOT IN ('2', '3')
),

prices_dedup AS (
    -- - if PT60M & PT15M exist → only PT60M
    -- - if only PT15M  → PT15M
    -- - if only PT60M  → PT60M
    SELECT *
    FROM prices_raw
    WHERE
        ResolutionCode = 'PT60M'
        OR (ResolutionCode = 'PT15M' AND cnt_60m_same_ts = 0)
),

-- 1. cumulative negative pricing hours from prices_dedup
neg_hours AS (
    SELECT
        AreaDisplayName AS country,
        SUM(
            CASE
                WHEN ResolutionCode = 'PT60M' AND "Price[Currency/MWh]" < 0 THEN 1.0
                WHEN ResolutionCode = 'PT15M' AND "Price[Currency/MWh]" < 0 THEN 0.25
                ELSE 0.0
            END
        ) AS neg_hours_total
    FROM prices_dedup
    GROUP BY AreaDisplayName
),

-- 2. Avg. market price )
avg_price AS (
    SELECT
        AreaDisplayName AS country,
        AVG("Price[Currency/MWh]") AS avg_price_eur_per_mwh
    FROM prices_dedup
    GROUP BY AreaDisplayName
),

-- 3. joining on time and area codes 
joined AS (
    SELECT
        ep.AreaDisplayName AS country,
        ep.AreaCode,
        ep."DateTime(UTC)",
        ep.ResolutionCode,

        -- set resolutions codes
        CASE
            WHEN ep.ResolutionCode = 'PT15M' THEN 0.25
            WHEN ep.ResolutionCode = 'PT60M' THEN 1.0
            ELSE 1.0
        END AS interval_hours,

        ep."Price[Currency/MWh]" AS price_raw,

        ag."ActualGenerationOutput" AS gen_mw
    FROM prices_dedup AS ep
    JOIN read_csv_auto(
            '2025_07_AggregatedGenerationPerType_16.1.B_C.csv',
            strict_mode=false,
            null_padding=true
        ) AS ag
      ON ep.AreaCode        = ag.AreaCode
     AND ep."DateTime(UTC)" = ag."DateTime"
    WHERE
        ag.ProductionType = 'Solar'
        AND ag."ActualGenerationOutput" > 0
),

-- 4. Solar generation during negative pricing hours (in MWh)
solar_energy AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours) AS solar_mwh_total,
        SUM(
            CASE
                WHEN price_raw < 0 THEN gen_mw * interval_hours
                ELSE 0
            END
        ) AS solar_mwh_at_neg_price
    FROM joined
    GROUP BY country
),

-- 5. Capture Price (€/MWh)
capture AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours * price_raw)
        / NULLIF(SUM(gen_mw * interval_hours), 0) AS capture_eur_per_mwh
    FROM joined
    GROUP BY country
),

-- 6. Capture Price with floor price of 0 (€/MWh)
capture_floor0 AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours * CASE WHEN price_raw < 0 THEN 0 ELSE price_raw END)
        / NULLIF(SUM(gen_mw * interval_hours), 0) AS capture_price_floor0_eur_per_mwh
    FROM joined
    GROUP BY country
)

SELECT
    a.country,
    n.neg_hours_total,

    -- avg market price electrcity (Day-ahead) in ct/kWh
    ROUND(a.avg_price_eur_per_mwh * 0.1, 3) AS avg_price_ct_per_kwh,

    -- Capture Price in ct/kWh
    ROUND(c.capture_eur_per_mwh * 0.1, 3)   AS capture_price_ct_per_kwh,
	
    -- Capture Price with floor price of 0 in ct/kWh
    ROUND(cf.capture_price_floor0_eur_per_mwh * 0.1, 3) AS capture_price_floor0_ct_kwh,

    -- Capture Rate in % 
    ROUND(
        100.0 * c.capture_eur_per_mwh
        / NULLIF(a.avg_price_eur_per_mwh, 0),
        2
    ) AS capture_rate_percent,

    -- volume share of solar power generated during negative pricing hours (in %)
    ROUND(
        100.0 * s.solar_mwh_at_neg_price
        / NULLIF(s.solar_mwh_total, 0),
        2
    ) AS share_solar_at_neg_price_percent

FROM avg_price a
LEFT JOIN neg_hours    n ON a.country = n.country
LEFT JOIN capture      c ON a.country = c.country
LEFT JOIN solar_energy s ON a.country = s.country
LEFT JOIN capture_floor0 cf ON a.country = cf.country
ORDER BY a.country;

,country,neg_hours_total,avg_price_ct_per_kwh,capture_price_ct_per_kwh,capture_price_floor0_ct_kwh,capture_rate_percent,share_solar_at_neg_price_percent
0,Austria (AT),6.0,8.791,5.858,5.865,66.64,3.66
1,Belgium (BE),11.0,8.306,5.746,5.755,69.18,3.53
2,Bulgaria (BG),1.0,10.112,7.379,7.379,72.97,0.21
3,Croatia (HR),5.0,10.303,7.841,7.841,76.10,1.00
4,Czech Republic (CZ),6.0,9.188,6.613,6.615,71.97,2.95
5,DE-LU,12.0,8.779,5.984,5.989,68.16,5.24
6,DK1,12.0,8.280,6.098,6.099,73.65,2.11
7,DK2,4.0,7.996,5.929,5.930,74.16,1.47
8,Estonia (EE),18.0,3.698,2.669,2.669,72.17,2.70
9,Finland (FI),18.0,2.418,2.596,2.596,107.36,2.32


In [27]:
WITH prices_raw AS (
    SELECT
        ep.*,
        COUNT(*) FILTER (
            WHERE ep.ResolutionCode = 'PT60M'
        ) OVER (
            PARTITION BY ep.AreaCode, ep."DateTime(UTC)"
        ) AS cnt_60m_same_ts
    FROM read_csv_auto(
            '2025_08_EnergyPrices_12.1.D_r3.csv',
            strict_mode=false,
            null_padding=true
        ) AS ep
    WHERE
        ep.ContractType = 'Day-ahead'
        AND ep.Sequence NOT IN ('2', '3')
),

prices_dedup AS (
    -- - if PT60M & PT15M exist → only PT60M
    -- - if only PT15M  → PT15M
    -- - if only PT60M  → PT60M
    SELECT *
    FROM prices_raw
    WHERE
        ResolutionCode = 'PT60M'
        OR (ResolutionCode = 'PT15M' AND cnt_60m_same_ts = 0)
),

-- 1. cumulative negative pricing hours from prices_dedup
neg_hours AS (
    SELECT
        AreaDisplayName AS country,
        SUM(
            CASE
                WHEN ResolutionCode = 'PT60M' AND "Price[Currency/MWh]" < 0 THEN 1.0
                WHEN ResolutionCode = 'PT15M' AND "Price[Currency/MWh]" < 0 THEN 0.25
                ELSE 0.0
            END
        ) AS neg_hours_total
    FROM prices_dedup
    GROUP BY AreaDisplayName
),

-- 2. Avg. market price )
avg_price AS (
    SELECT
        AreaDisplayName AS country,
        AVG("Price[Currency/MWh]") AS avg_price_eur_per_mwh
    FROM prices_dedup
    GROUP BY AreaDisplayName
),

-- 3. joining on time and area codes 
joined AS (
    SELECT
        ep.AreaDisplayName AS country,
        ep.AreaCode,
        ep."DateTime(UTC)",
        ep.ResolutionCode,

        -- set resolutions codes
        CASE
            WHEN ep.ResolutionCode = 'PT15M' THEN 0.25
            WHEN ep.ResolutionCode = 'PT60M' THEN 1.0
            ELSE 1.0
        END AS interval_hours,

        ep."Price[Currency/MWh]" AS price_raw,

        ag."ActualGenerationOutput" AS gen_mw
    FROM prices_dedup AS ep
    JOIN read_csv_auto(
            '2025_08_AggregatedGenerationPerType_16.1.B_C.csv',
            strict_mode=false,
            null_padding=true
        ) AS ag
      ON ep.AreaCode        = ag.AreaCode
     AND ep."DateTime(UTC)" = ag."DateTime"
    WHERE
        ag.ProductionType = 'Solar'
        AND ag."ActualGenerationOutput" > 0
),

-- 4. Solar generation during negative pricing hours (in MWh)
solar_energy AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours) AS solar_mwh_total,
        SUM(
            CASE
                WHEN price_raw < 0 THEN gen_mw * interval_hours
                ELSE 0
            END
        ) AS solar_mwh_at_neg_price
    FROM joined
    GROUP BY country
),

-- 5. Capture Price (€/MWh)
capture AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours * price_raw)
        / NULLIF(SUM(gen_mw * interval_hours), 0) AS capture_eur_per_mwh
    FROM joined
    GROUP BY country
),

-- 6. Capture Price with floor price of 0 (€/MWh)
capture_floor0 AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours * CASE WHEN price_raw < 0 THEN 0 ELSE price_raw END)
        / NULLIF(SUM(gen_mw * interval_hours), 0) AS capture_price_floor0_eur_per_mwh
    FROM joined
    GROUP BY country
)

SELECT
    a.country,
    n.neg_hours_total,

    -- avg market price electrcity (Day-ahead) in ct/kWh
    ROUND(a.avg_price_eur_per_mwh * 0.1, 3) AS avg_price_ct_per_kwh,

    -- Capture Price in ct/kWh
    ROUND(c.capture_eur_per_mwh * 0.1, 3)   AS capture_price_ct_per_kwh,
	
    -- Capture Price with floor price of 0 in ct/kWh
    ROUND(cf.capture_price_floor0_eur_per_mwh * 0.1, 3) AS capture_price_floor0_ct_kwh,

    -- Capture Rate in % 
    ROUND(
        100.0 * c.capture_eur_per_mwh
        / NULLIF(a.avg_price_eur_per_mwh, 0),
        2
    ) AS capture_rate_percent,

    -- volume share of solar power generated during negative pricing hours (in %)
    ROUND(
        100.0 * s.solar_mwh_at_neg_price
        / NULLIF(s.solar_mwh_total, 0),
        2
    ) AS share_solar_at_neg_price_percent

FROM avg_price a
LEFT JOIN neg_hours    n ON a.country = n.country
LEFT JOIN capture      c ON a.country = c.country
LEFT JOIN solar_energy s ON a.country = s.country
LEFT JOIN capture_floor0 cf ON a.country = cf.country
ORDER BY a.country;

,country,neg_hours_total,avg_price_ct_per_kwh,capture_price_ct_per_kwh,capture_price_floor0_ct_kwh,capture_rate_percent,share_solar_at_neg_price_percent
0,Austria (AT),59.0,7.426,3.195,3.419,43.02,23.89
1,Belgium (BE),58.0,6.889,3.405,3.614,49.42,21.92
2,Bulgaria (BG),32.0,7.579,4.079,4.128,53.81,6.71
3,Croatia (HR),43.0,7.862,4.093,4.163,52.06,9.11
4,Czech Republic (CZ),54.0,7.700,3.891,4.027,50.53,19.22
5,DE-LU,64.0,7.694,3.942,4.084,51.23,22.12
6,DK1,46.0,7.342,4.694,4.745,63.93,9.86
7,DK2,31.0,7.526,4.229,4.267,56.18,10.80
8,Estonia (EE),20.0,7.729,5.484,5.488,70.95,3.63
9,Finland (FI),37.0,5.557,5.426,5.430,97.65,4.34


In [25]:
WITH prices_raw AS (
    SELECT
        ep.*,
        COUNT(*) FILTER (
            WHERE ep.ResolutionCode = 'PT60M'
        ) OVER (
            PARTITION BY ep.AreaCode, ep."DateTime(UTC)"
        ) AS cnt_60m_same_ts
    FROM read_csv_auto(
            '2025_09_EnergyPrices_12.1.D_r3.csv',
            strict_mode=false,
            null_padding=true
        ) AS ep
    WHERE
        ep.ContractType = 'Day-ahead'
        AND ep.Sequence NOT IN ('2', '3')
),

prices_dedup AS (
    -- - if PT60M & PT15M exist → only PT60M
    -- - if only PT15M  → PT15M
    -- - if only PT60M  → PT60M
    SELECT *
    FROM prices_raw
    WHERE
        ResolutionCode = 'PT60M'
        OR (ResolutionCode = 'PT15M' AND cnt_60m_same_ts = 0)
),

-- 1. cumulative negative pricing hours from prices_dedup
neg_hours AS (
    SELECT
        AreaDisplayName AS country,
        SUM(
            CASE
                WHEN ResolutionCode = 'PT60M' AND "Price[Currency/MWh]" < 0 THEN 1.0
                WHEN ResolutionCode = 'PT15M' AND "Price[Currency/MWh]" < 0 THEN 0.25
                ELSE 0.0
            END
        ) AS neg_hours_total
    FROM prices_dedup
    GROUP BY AreaDisplayName
),

-- 2. Avg. market price )
avg_price AS (
    SELECT
        AreaDisplayName AS country,
        AVG("Price[Currency/MWh]") AS avg_price_eur_per_mwh
    FROM prices_dedup
    GROUP BY AreaDisplayName
),

-- 3. joining on time and area codes 
joined AS (
    SELECT
        ep.AreaDisplayName AS country,
        ep.AreaCode,
        ep."DateTime(UTC)",
        ep.ResolutionCode,

        -- set resolutions codes
        CASE
            WHEN ep.ResolutionCode = 'PT15M' THEN 0.25
            WHEN ep.ResolutionCode = 'PT60M' THEN 1.0
            ELSE 1.0
        END AS interval_hours,

        ep."Price[Currency/MWh]" AS price_raw,

        ag."ActualGenerationOutput" AS gen_mw
    FROM prices_dedup AS ep
    JOIN read_csv_auto(
            '2025_09_AggregatedGenerationPerType_16.1.B_C.csv',
            strict_mode=false,
            null_padding=true
        ) AS ag
      ON ep.AreaCode        = ag.AreaCode
     AND ep."DateTime(UTC)" = ag."DateTime"
    WHERE
        ag.ProductionType = 'Solar'
        AND ag."ActualGenerationOutput" > 0
),

-- 4. Solar generation during negative pricing hours (in MWh)
solar_energy AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours) AS solar_mwh_total,
        SUM(
            CASE
                WHEN price_raw < 0 THEN gen_mw * interval_hours
                ELSE 0
            END
        ) AS solar_mwh_at_neg_price
    FROM joined
    GROUP BY country
),

-- 5. Capture Price (€/MWh)
capture AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours * price_raw)
        / NULLIF(SUM(gen_mw * interval_hours), 0) AS capture_eur_per_mwh
    FROM joined
    GROUP BY country
),

-- 6. Capture Price with floor price of 0 (€/MWh)
capture_floor0 AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours * CASE WHEN price_raw < 0 THEN 0 ELSE price_raw END)
        / NULLIF(SUM(gen_mw * interval_hours), 0) AS capture_price_floor0_eur_per_mwh
    FROM joined
    GROUP BY country
)

SELECT
    a.country,
    n.neg_hours_total,

    -- avg market price electrcity (Day-ahead) in ct/kWh
    ROUND(a.avg_price_eur_per_mwh * 0.1, 3) AS avg_price_ct_per_kwh,

    -- Capture Price in ct/kWh
    ROUND(c.capture_eur_per_mwh * 0.1, 3)   AS capture_price_ct_per_kwh,
	
    -- Capture Price with floor price of 0 in ct/kWh
    ROUND(cf.capture_price_floor0_eur_per_mwh * 0.1, 3) AS capture_price_floor0_ct_kwh,

    -- Capture Rate in % 
    ROUND(
        100.0 * c.capture_eur_per_mwh
        / NULLIF(a.avg_price_eur_per_mwh, 0),
        2
    ) AS capture_rate_percent,

    -- volume share of solar power generated during negative pricing hours (in %)
    ROUND(
        100.0 * s.solar_mwh_at_neg_price
        / NULLIF(s.solar_mwh_total, 0),
        2
    ) AS share_solar_at_neg_price_percent

FROM avg_price a
LEFT JOIN neg_hours    n ON a.country = n.country
LEFT JOIN capture      c ON a.country = c.country
LEFT JOIN solar_energy s ON a.country = s.country
LEFT JOIN capture_floor0 cf ON a.country = cf.country
ORDER BY a.country;

,country,neg_hours_total,avg_price_ct_per_kwh,capture_price_ct_per_kwh,capture_price_floor0_ct_kwh,capture_rate_percent,share_solar_at_neg_price_percent
0,Austria (AT),23.0,9.233,4.857,5.030,52.60,13.96
1,Belgium (BE),58.0,6.698,3.634,3.816,54.26,21.36
2,Bulgaria (BG),22.0,9.376,5.984,6.024,63.82,4.59
3,Croatia (HR),23.0,9.571,6.549,6.607,68.43,5.27
4,Czech Republic (CZ),23.0,9.214,5.672,5.829,61.56,12.54
5,DE-LU,60.0,8.357,4.452,4.648,53.27,24.78
6,DK1,27.0,7.256,4.821,4.862,66.45,5.20
7,DK2,5.0,8.335,5.190,5.191,62.27,1.01
8,Estonia (EE),7.0,8.114,6.439,6.440,79.36,1.14
9,Finland (FI),19.0,4.147,4.708,4.708,113.52,1.06


In [26]:
WITH prices_raw AS (
    SELECT
        ep.*,
        COUNT(*) FILTER (
            WHERE ep.ResolutionCode = 'PT60M'
        ) OVER (
            PARTITION BY ep.AreaCode, ep."DateTime(UTC)"
        ) AS cnt_60m_same_ts
    FROM read_csv_auto(
            '2025_10_EnergyPrices_12.1.D_r3.csv',
            strict_mode=false,
            null_padding=true
        ) AS ep
    WHERE
        ep.ContractType = 'Day-ahead'
        AND ep.Sequence NOT IN ('2', '3')
),

prices_dedup AS (
    -- - if PT60M & PT15M exist → only PT60M
    -- - if only PT15M  → PT15M
    -- - if only PT60M  → PT60M
    SELECT *
    FROM prices_raw
    WHERE
        ResolutionCode = 'PT60M'
        OR (ResolutionCode = 'PT15M' AND cnt_60m_same_ts = 0)
),

-- 1. cumulative negative pricing hours from prices_dedup
neg_hours AS (
    SELECT
        AreaDisplayName AS country,
        SUM(
            CASE
                WHEN ResolutionCode = 'PT60M' AND "Price[Currency/MWh]" < 0 THEN 1.0
                WHEN ResolutionCode = 'PT15M' AND "Price[Currency/MWh]" < 0 THEN 0.25
                ELSE 0.0
            END
        ) AS neg_hours_total
    FROM prices_dedup
    GROUP BY AreaDisplayName
),

-- 2. Avg. market price )
avg_price AS (
    SELECT
        AreaDisplayName AS country,
        AVG("Price[Currency/MWh]") AS avg_price_eur_per_mwh
    FROM prices_dedup
    GROUP BY AreaDisplayName
),

-- 3. joining on time and area codes 
joined AS (
    SELECT
        ep.AreaDisplayName AS country,
        ep.AreaCode,
        ep."DateTime(UTC)",
        ep.ResolutionCode,

        -- set resolutions codes
        CASE
            WHEN ep.ResolutionCode = 'PT15M' THEN 0.25
            WHEN ep.ResolutionCode = 'PT60M' THEN 1.0
            ELSE 1.0
        END AS interval_hours,

        ep."Price[Currency/MWh]" AS price_raw,

        ag."ActualGenerationOutput" AS gen_mw
    FROM prices_dedup AS ep
    JOIN read_csv_auto(
            '2025_10_AggregatedGenerationPerType_16.1.B_C.csv',
            strict_mode=false,
            null_padding=true
        ) AS ag
      ON ep.AreaCode        = ag.AreaCode
     AND ep."DateTime(UTC)" = ag."DateTime"
    WHERE
        ag.ProductionType = 'Solar'
        AND ag."ActualGenerationOutput" > 0
),

-- 4. Solar generation during negative pricing hours (in MWh)
solar_energy AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours) AS solar_mwh_total,
        SUM(
            CASE
                WHEN price_raw < 0 THEN gen_mw * interval_hours
                ELSE 0
            END
        ) AS solar_mwh_at_neg_price
    FROM joined
    GROUP BY country
),

-- 5. Capture Price (€/MWh)
capture AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours * price_raw)
        / NULLIF(SUM(gen_mw * interval_hours), 0) AS capture_eur_per_mwh
    FROM joined
    GROUP BY country
),

-- 6. Capture Price with floor price of 0 (€/MWh)
capture_floor0 AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours * CASE WHEN price_raw < 0 THEN 0 ELSE price_raw END)
        / NULLIF(SUM(gen_mw * interval_hours), 0) AS capture_price_floor0_eur_per_mwh
    FROM joined
    GROUP BY country
)

SELECT
    a.country,
    n.neg_hours_total,

    -- avg market price electrcity (Day-ahead) in ct/kWh
    ROUND(a.avg_price_eur_per_mwh * 0.1, 3) AS avg_price_ct_per_kwh,

    -- Capture Price in ct/kWh
    ROUND(c.capture_eur_per_mwh * 0.1, 3)   AS capture_price_ct_per_kwh,
	
    -- Capture Price with floor price of 0 in ct/kWh
    ROUND(cf.capture_price_floor0_eur_per_mwh * 0.1, 3) AS capture_price_floor0_ct_kwh,

    -- Capture Rate in % 
    ROUND(
        100.0 * c.capture_eur_per_mwh
        / NULLIF(a.avg_price_eur_per_mwh, 0),
        2
    ) AS capture_rate_percent,

    -- volume share of solar power generated during negative pricing hours (in %)
    ROUND(
        100.0 * s.solar_mwh_at_neg_price
        / NULLIF(s.solar_mwh_total, 0),
        2
    ) AS share_solar_at_neg_price_percent

FROM avg_price a
LEFT JOIN neg_hours    n ON a.country = n.country
LEFT JOIN capture      c ON a.country = c.country
LEFT JOIN solar_energy s ON a.country = s.country
LEFT JOIN capture_floor0 cf ON a.country = cf.country
ORDER BY a.country;

,country,neg_hours_total,avg_price_ct_per_kwh,capture_price_ct_per_kwh,capture_price_floor0_ct_kwh,capture_rate_percent,share_solar_at_neg_price_percent
0,Austria (AT),0.00,10.893,8.977,8.977,82.41,0.00
1,Belgium (BE),32.25,7.499,6.643,6.659,88.59,12.23
2,Bulgaria (BG),0.00,12.180,11.671,11.671,95.82,0.00
3,Croatia (HR),0.25,11.315,9.565,9.565,84.53,0.03
4,Czech Republic (CZ),8.00,9.565,8.273,8.273,86.49,0.07
5,DE-LU,49.75,8.435,7.691,7.704,91.18,7.11
6,DK1,30.00,7.815,7.535,7.541,96.41,3.09
7,DK2,15.00,8.210,7.655,7.655,93.24,0.00
8,Estonia (EE),11.25,8.969,7.686,7.686,85.70,0.03
9,Finland (FI),21.75,4.898,5.751,5.751,117.41,0.07


In [31]:
WITH prices_raw AS (
    SELECT
        ep.*,
        COUNT(*) FILTER (
            WHERE ep.ResolutionCode = 'PT60M'
        ) OVER (
            PARTITION BY ep.AreaCode, ep."DateTime(UTC)"
        ) AS cnt_60m_same_ts
    FROM read_csv_auto(
            '2025_11_EnergyPrices_12.1.D_r3.csv',
            strict_mode=false,
            null_padding=true
        ) AS ep
    WHERE
        ep.ContractType = 'Day-ahead'
        AND ep.Sequence NOT IN ('2', '3')
),

prices_dedup AS (
    -- - if PT60M & PT15M exist → only PT60M
    -- - if only PT15M  → PT15M
    -- - if only PT60M  → PT60M
    SELECT *
    FROM prices_raw
    WHERE
        ResolutionCode = 'PT60M'
        OR (ResolutionCode = 'PT15M' AND cnt_60m_same_ts = 0)
),

-- 1. cumulative negative pricing hours from prices_dedup
neg_hours AS (
    SELECT
        AreaDisplayName AS country,
        SUM(
            CASE
                WHEN ResolutionCode = 'PT60M' AND "Price[Currency/MWh]" < 0 THEN 1.0
                WHEN ResolutionCode = 'PT15M' AND "Price[Currency/MWh]" < 0 THEN 0.25
                ELSE 0.0
            END
        ) AS neg_hours_total
    FROM prices_dedup
    GROUP BY AreaDisplayName
),

-- 2. Avg. market price )
avg_price AS (
    SELECT
        AreaDisplayName AS country,
        AVG("Price[Currency/MWh]") AS avg_price_eur_per_mwh
    FROM prices_dedup
    GROUP BY AreaDisplayName
),

-- 3. joining on time and area codes 
joined AS (
    SELECT
        ep.AreaDisplayName AS country,
        ep.AreaCode,
        ep."DateTime(UTC)",
        ep.ResolutionCode,

        -- set resolutions codes
        CASE
            WHEN ep.ResolutionCode = 'PT15M' THEN 0.25
            WHEN ep.ResolutionCode = 'PT60M' THEN 1.0
            ELSE 1.0
        END AS interval_hours,

        ep."Price[Currency/MWh]" AS price_raw,

        ag."ActualGenerationOutput[MW]" AS gen_mw
    FROM prices_dedup AS ep
    JOIN read_csv_auto(
            '2025_11_AggregatedGenerationPerType_16.1.B_C_r3.csv',
            strict_mode=false,
            null_padding=true
        ) AS ag
      ON ep.AreaCode        = ag.AreaCode
     AND ep."DateTime(UTC)" = ag."DateTime(UTC)"
    WHERE
        ag.ProductionType = 'Solar'
        AND ag."ActualGenerationOutput[MW]" > 0
),

-- 4. Solar generation during negative pricing hours (in MWh)
solar_energy AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours) AS solar_mwh_total,
        SUM(
            CASE
                WHEN price_raw < 0 THEN gen_mw * interval_hours
                ELSE 0
            END
        ) AS solar_mwh_at_neg_price
    FROM joined
    GROUP BY country
),

-- 5. Capture Price (€/MWh)
capture AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours * price_raw)
        / NULLIF(SUM(gen_mw * interval_hours), 0) AS capture_eur_per_mwh
    FROM joined
    GROUP BY country
),

-- 6. Capture Price with floor price of 0 (€/MWh)
capture_floor0 AS (
    SELECT
        country,
        SUM(gen_mw * interval_hours * CASE WHEN price_raw < 0 THEN 0 ELSE price_raw END)
        / NULLIF(SUM(gen_mw * interval_hours), 0) AS capture_price_floor0_eur_per_mwh
    FROM joined
    GROUP BY country
)

SELECT
    a.country,
    n.neg_hours_total,

    -- avg market price electrcity (Day-ahead) in ct/kWh
    ROUND(a.avg_price_eur_per_mwh * 0.1, 3) AS avg_price_ct_per_kwh,

    -- Capture Price in ct/kWh
    ROUND(c.capture_eur_per_mwh * 0.1, 3)   AS capture_price_ct_per_kwh,
	
    -- Capture Price with floor price of 0 in ct/kWh
    ROUND(cf.capture_price_floor0_eur_per_mwh * 0.1, 3) AS capture_price_floor0_ct_kwh,

    -- Capture Rate in % 
    ROUND(
        100.0 * c.capture_eur_per_mwh
        / NULLIF(a.avg_price_eur_per_mwh, 0),
        2
    ) AS capture_rate_percent,

    -- volume share of solar power generated during negative pricing hours (in %)
    ROUND(
        100.0 * s.solar_mwh_at_neg_price
        / NULLIF(s.solar_mwh_total, 0),
        2
    ) AS share_solar_at_neg_price_percent

FROM avg_price a
LEFT JOIN neg_hours    n ON a.country = n.country
LEFT JOIN capture      c ON a.country = c.country
LEFT JOIN solar_energy s ON a.country = s.country
LEFT JOIN capture_floor0 cf ON a.country = cf.country
ORDER BY a.country;

,country,neg_hours_total,avg_price_ct_per_kwh,capture_price_ct_per_kwh,capture_price_floor0_ct_kwh,capture_rate_percent,share_solar_at_neg_price_percent
0,Austria (AT),0.00,11.604,10.135,10.135,87.34,0.00
1,Belgium (BE),0.00,8.626,7.854,7.854,91.05,0.00
2,Bulgaria (BG),0.00,12.151,11.392,11.392,93.75,0.00
3,Croatia (HR),0.00,11.826,10.608,10.608,89.70,0.00
4,Czech Republic (CZ),0.25,11.153,9.666,9.666,86.66,0.00
5,DE-LU,0.00,10.193,9.089,9.089,89.17,0.00
6,DK1,0.00,9.289,9.318,9.318,100.31,0.00
7,DK2,0.00,9.365,9.059,9.059,96.74,0.00
8,Estonia (EE),7.25,9.585,11.241,11.241,117.28,0.04
9,Finland (FI),10.00,4.790,5.713,5.713,119.27,0.11


In [30]:
SELECT * FROM read_csv_auto('2025_06_AggregatedGenerationPerType_16.1.B_C.csv') LIMIT 5;

,DateTime,ResolutionCode,AreaCode,AreaTypeCode,AreaName,MapCode,ProductionType,ActualGenerationOutput,ActualConsumption,UpdateTime
0,2025-06-01 05:00:00+00:00,PT60M,10YCH-SWISSGRIDZ,CTA,CH CTA,CH,Solar,582.42,NaN,2025-06-02 08:04:05.005000+00:00
1,2025-06-01 06:00:00+00:00,PT60M,10YCH-SWISSGRIDZ,CTA,CH CTA,CH,Solar,1190.51,NaN,2025-06-02 08:04:05.005000+00:00
2,2025-06-01 07:00:00+00:00,PT60M,10YCH-SWISSGRIDZ,CTA,CH CTA,CH,Solar,2073.43,NaN,2025-06-02 08:04:05.005000+00:00
3,2025-06-01 08:00:00+00:00,PT60M,10YCH-SWISSGRIDZ,CTA,CH CTA,CH,Solar,2661.50,NaN,2025-06-02 08:04:05.005000+00:00
4,2025-06-01 18:00:00+00:00,PT60M,10YCH-SWISSGRIDZ,CTA,CH CTA,CH,Solar,131.38,NaN,2025-06-01 21:33:43.043000+00:00


In [17]:
WITH prices_raw AS (
    SELECT
        ep.*,
        COUNT(*) FILTER (
            WHERE ResolutionCode = 'PT60M'
        ) OVER (
            PARTITION BY AreaCode, "DateTime(UTC)"
        ) AS cnt_60m_same_ts
    FROM read_csv_auto('2025_04_EnergyPrices_12.1.D_r3.csv') ep
    WHERE
        ep.ContractType = 'Day-ahead'
        AND ep.Sequence NOT IN ('2', '3')
),

prices_dedup AS (
    -- PT60M gewinnt, PT15M nur wenn kein PT60M für den Timestamp existiert
    SELECT *
    FROM prices_raw
    WHERE
        ResolutionCode = 'PT60M'
        OR (ResolutionCode = 'PT15M' AND cnt_60m_same_ts = 0)
),

-- Basis: alle Märkte mit Day-ahead-Preisen (für negative Stunden & avg price)
market_stats AS (
    SELECT
        AreaDisplayName AS country,
        -- kumulierte negative Stunden
        SUM(
            CASE
                WHEN "Price[Currency/MWh]" < 0 AND ResolutionCode = 'PT60M' THEN 1.0
                WHEN "Price[Currency/MWh]" < 0 AND ResolutionCode = 'PT15M' THEN 0.25
                ELSE 0.0
            END
        ) AS neg_hours,
        -- durchschnittlicher Marktpreis (€/MWh)
        AVG("Price[Currency/MWh]") AS avg_price_eur_mwh
    FROM prices_dedup
    GROUP BY AreaDisplayName
),

-- sauberer zeitlicher Join Preise + Solar
joined AS (
    SELECT
        ep.AreaDisplayName AS country,
        ep."DateTime(UTC)",
        -- Preis für Capture-Price-Berechnung (negative auf 0 setzen)
        CASE
            WHEN ep."Price[Currency/MWh]" < 0 THEN 0
            ELSE ep."Price[Currency/MWh]"
        END AS price_nonneg_eur_mwh,
        ag."ActualGenerationOutput[MW]" AS gen_mw,
        -- Flag, ob Preis negativ war (für "Solar bei negativen Preisen")
        CASE
            WHEN ep."Price[Currency/MWh]" < 0 THEN 1
            ELSE 0
        END AS is_negative_price
    FROM prices_dedup ep
    JOIN (
        SELECT * FROM read_csv_auto('2025_04_AggregatedGenerationPerType_16.1.B_C.csv')
    ) AS ag1
      ON ep.AreaCode        = ag1.AreaCode
     AND ep."DateTime(UTC)" = ag1."DateTime(UTC)"
    WHERE
        ag1.ProductionType = 'Solar'
        AND ag1."ActualGenerationOutput[MW]" > 0
),

-- Capture Price nach der "ct/kWh"-Logik, die gut zu Netztransparenz passt
capture AS (
    SELECT
        country,
        SUM(gen_mw * price_nonneg_eur_mwh) AS num_mw_eur_per_mwh,
        SUM(gen_mw)                         AS den_mw,
        -- Solar-Mengen für Anteil bei negativen Preisen
        SUM(CASE WHEN is_negative_price = 1 THEN gen_mw ELSE 0 END) AS gen_mw_neg,
        SUM(gen_mw)                                                   AS gen_mw_total
    FROM joined
    GROUP BY country
),

capture_final AS (
    SELECT
        country,
        -- €/MWh → ct/kWh : /10
        CASE 
            WHEN den_mw > 0 THEN (num_mw_eur_per_mwh / den_mw) / 10
            ELSE NULL
        END AS capture_price_ct_per_kwh,
        CASE 
            WHEN gen_mw_total > 0 THEN 100.0 * gen_mw_neg / gen_mw_total
            ELSE NULL
        END AS share_solar_neg_pct
    FROM capture
)

SELECT
    ms.country,
    -- 1) kumulierte negative Stunden
    ms.neg_hours,
    -- 2) durchschnittlicher Marktpreis (€/MWh) + in ct/kWh
    ROUND(ms.avg_price_eur_mwh, 2) AS avg_price_eur_mwh,
    ROUND(ms.avg_price_eur_mwh / 10, 3) AS avg_price_ct_per_kwh,
    -- 3) Capture Price (ct/kWh) nach "Netztransparenz-naher" Logik
    ROUND(cf.capture_price_ct_per_kwh, 3) AS capture_price_ct_per_kwh,
    -- 4) Capture Rate in % (nicht 0,43 sondern 43,0)
    CASE
        WHEN cf.capture_price_ct_per_kwh IS NOT NULL
             AND ms.avg_price_eur_mwh IS NOT NULL
             AND ms.avg_price_eur_mwh <> 0
        THEN ROUND(
            (cf.capture_price_ct_per_kwh / (ms.avg_price_eur_mwh / 10)) * 100
        , 1)
        ELSE NULL
    END AS capture_rate_pct,
    -- 5) Anteil der Solar-Erzeugung bei negativen Preisen (%)
    ROUND(cf.share_solar_neg_pct, 2) AS share_solar_at_negative_prices_pct
FROM market_stats ms
LEFT JOIN capture_final cf
       ON ms.country = cf.country
ORDER BY ms.country;

Error: Binder Error: Referenced table "ag1" not found!
Candidate tables: "ms"

LINE 63:      AND ep."DateTime(UTC)" = ag1."DateTime(UTC)"
                                       ^

In [8]:
SELECT AreaDisplayName, ContractType
FROM '2025_10_EnergyPrices_12.1.D_r3.csv'
GROUP BY AreaDisplayName, ContractType;

,AreaDisplayName,ContractType
0,Portugal (PT),Day-ahead
1,SE3,Day-ahead
2,Slovakia (SK),Day-ahead
3,Switzerland (CH),Day-ahead
4,DE-LU,Day-ahead
5,Spain (ES),Intraday
6,France (FR),Day-ahead
7,IT-Calabria,Day-ahead
8,IT-North,Day-ahead
9,IT-Sardinia,Day-ahead


In [8]:
WITH area_resolution AS (
    SELECT
        AreaDisplayName,
        MAX(CASE WHEN ResolutionCode = 'PT60M' THEN 1 ELSE 0 END) AS has_PT60M,
        MAX(CASE WHEN ResolutionCode = 'PT15M' THEN 1 ELSE 0 END) AS has_PT15M
    FROM (SELECT AreaDisplayName, ResolutionCode FROM '2025_11_EnergyPrices_12.1.D_r3.csv')
    GROUP BY AreaDisplayName
),
filtered_data AS (
    SELECT
        d.AreaDisplayName,
        d."Price[Currency/MWh]",
        d.ResolutionCode,
        d.ContractType,
        d.Sequence,
        ar.has_PT60M,
        ar.has_PT15M
    FROM '2025_11_EnergyPrices_12.1.D_r3.csv' d
    JOIN area_resolution ar ON d.AreaDisplayName = ar.AreaDisplayName
    WHERE
        d.ContractType = 'Day-ahead'
        AND (d.Sequence NOT LIKE '2' AND d.Sequence NOT LIKE '3')
        -- Only keep PT60M rows if both PT60M and PT15M are available
        AND (
            (ar.has_PT60M = 1 AND ar.has_PT15M = 1 AND d.ResolutionCode = 'PT60M')
            OR (ar.has_PT60M = 1 AND ar.has_PT15M = 0 AND d.ResolutionCode = 'PT60M')
            OR (ar.has_PT60M = 0 AND ar.has_PT15M = 1 AND d.ResolutionCode = 'PT15M')
        )
)
SELECT
    AreaDisplayName AS Country,
    SUM(
        CASE
            WHEN ResolutionCode = 'PT60M' AND "Price[Currency/MWh]" < 0 THEN 1
            WHEN ResolutionCode = 'PT15M' AND "Price[Currency/MWh]" < 0 THEN 0.25
            ELSE 0
        END
    ) AS total_hours
FROM filtered_data
GROUP BY AreaDisplayName
ORDER BY total_hours DESC;

,Country,total_hours
0,SE2,74.50
1,SE4,11.50
2,SE3,9.75
3,Finland (FI),5.00
4,Estonia (EE),4.50
5,Spain (ES),4.25
6,Lithuania (LT),3.50
7,Latvia (LV),3.50
8,SE1,2.75
9,Poland (PL),1.00


In [3]:
SELECT
    name,
    has_PT60M,
    has_PT15M,
    CASE
        WHEN has_PT60M = 1 AND has_PT15M = 1 THEN 1
        ELSE 0
    END AS has_both
FROM (
    SELECT
        AreaDisplayName AS name,
        MAX(CASE WHEN ResolutionCode = 'PT60M' THEN 1 ELSE 0 END) AS has_PT60M,
        MAX(CASE WHEN ResolutionCode = 'PT15M' THEN 1 ELSE 0 END) AS has_PT15M
    FROM '2025_06_EnergyPrices_12.1.D_r3.csv'
    GROUP BY AreaDisplayName
)

,name,has_PT60M,has_PT15M,has_both
0,NO1,0,1,0
1,NO2NSL,1,0,0
2,Slovakia (SK),1,0,0
3,DK2,1,0,0
4,Hungary (HU),1,0,0
5,Montenegro (ME),1,0,0
6,NO4,0,1,0
7,NO5,0,1,0
8,Portugal (PT),1,0,0
9,SE3,1,0,0


In [11]:
WITH raw AS (
    SELECT
        *,
        CAST(SUBSTR(filename, 6, 2) AS INTEGER) AS month,
        COUNT(*) FILTER (
            WHERE ResolutionCode = 'PT15M'
        ) OVER (
            PARTITION BY AreaDisplayName, "DateTime(UTC)"
        ) AS cnt_15m_same_ts
    FROM read_csv_auto(
        '2025_??_EnergyPrices_12.1.D_r3.csv', 
        filename=true
    )
    WHERE 
        ContractType = 'Day-ahead'
        AND Sequence NOT IN ('2', '3')
),
agg AS (
    SELECT
        AreaDisplayName AS Country,
        month,
        SUM(
            CASE
                WHEN ResolutionCode = 'PT15M'
                     AND "Price[Currency/MWh]" < 0
                THEN 0.25
                ELSE 0
            END
        ) AS hours_15m,
        SUM(
            CASE
                WHEN ResolutionCode = 'PT60M'
                     AND cnt_15m_same_ts = 0
                     AND "Price[Currency/MWh]" < 0
                THEN 1
                ELSE 0
            END
        ) AS hours_60m
    FROM raw
    GROUP BY AreaDisplayName, month
),
agg_pivot AS (
    SELECT
        Country,
        COALESCE(SUM(CASE WHEN month = 1 THEN hours_15m + hours_60m END), 0) AS Jan_neg_hour,
        COALESCE(SUM(CASE WHEN month = 2 THEN hours_15m + hours_60m END), 0) AS Feb_neg_hour,
        COALESCE(SUM(CASE WHEN month = 3 THEN hours_15m + hours_60m END), 0) AS Mar_neg_hour,
        COALESCE(SUM(CASE WHEN month = 4 THEN hours_15m + hours_60m END), 0) AS Apr_neg_hour,
        COALESCE(SUM(CASE WHEN month = 5 THEN hours_15m + hours_60m END), 0) AS May_neg_hour,
        COALESCE(SUM(CASE WHEN month = 6 THEN hours_15m + hours_60m END), 0) AS Jun_neg_hour,
        COALESCE(SUM(CASE WHEN month = 7 THEN hours_15m + hours_60m END), 0) AS Jul_neg_hour,
        COALESCE(SUM(CASE WHEN month = 8 THEN hours_15m + hours_60m END), 0) AS Aug_neg_hour,
        COALESCE(SUM(CASE WHEN month = 9 THEN hours_15m + hours_60m END), 0) AS Sep_neg_hour,
        COALESCE(SUM(CASE WHEN month = 10 THEN hours_15m + hours_60m END), 0) AS Oct_neg_hour,
        COALESCE(SUM(CASE WHEN month = 11 THEN hours_15m + hours_60m END), 0) AS Nov_neg_hour,
        COALESCE(SUM(CASE WHEN month = 12 THEN hours_15m + hours_60m END), 0) AS Dec_neg_hour,
        COALESCE(SUM(hours_15m + hours_60m), 0) AS Annual_neg_hour
    FROM agg
    GROUP BY Country
),
prices_raw AS (
    SELECT
        ep.*,
        CAST(SUBSTR(ep.filename, 6, 2) AS INTEGER) AS month,
        COUNT(*) FILTER (
            WHERE ResolutionCode = 'PT60M'
        ) OVER (
            PARTITION BY AreaDisplayName, "DateTime(UTC)"
        ) AS cnt_60m_same_ts
    FROM read_csv_auto('2025_??_EnergyPrices_12.1.D_r3.csv', filename=true) AS ep
    WHERE
        ep.ContractType = 'Day-ahead'
        AND ep.Sequence NOT IN ('2', '3')
),
prices_dedup AS (
    SELECT *
    FROM prices_raw
    WHERE
        ResolutionCode = 'PT60M'
        OR (ResolutionCode = 'PT15M' AND cnt_60m_same_ts = 0)
),
joined AS (
    SELECT
        ep.AreaDisplayName AS country,
        ep."DateTime(UTC)",
        ep.ResolutionCode,
        ep.month,
        CASE
            WHEN ep."Price[Currency/MWh]" < 0 THEN 0
            ELSE ep."Price[Currency/MWh]"
        END AS price_nonneg,
        ep."Price[Currency/MWh]" AS price_raw,  -- FIX: Add the raw price column
        ag."ActualGenerationOutput" AS gen_mw,
        CASE
            WHEN ep.ResolutionCode = 'PT15M' THEN 0.25
            WHEN ep.ResolutionCode = 'PT60M' THEN 1.0
            ELSE 1.0
        END AS interval_hours
    FROM prices_dedup AS ep
    JOIN read_csv_auto(
            '2025_??_AggregatedGenerationPerType_16.1.B_C.csv',
            strict_mode=false,
            ignore_errors=true,
            filename=true
        ) AS ag
      ON ep.AreaCode = ag.AreaCode
     AND ep."DateTime(UTC)" = ag."DateTime"
     AND ep.month = CAST(SUBSTR(ag.filename, 6, 2) AS INTEGER)
    WHERE
        ag.ProductionType = 'Solar'
        AND ag."ActualGenerationOutput" > 0
),
capture_price AS (
    SELECT
        country,
        month,
        SUM(gen_mw * interval_hours * price_raw) AS weighted_price_sum,
        SUM(gen_mw * interval_hours) AS weighted_gen_sum,
        ROUND(
            SUM(gen_mw * interval_hours * price_raw)
            / NULLIF(SUM(gen_mw * interval_hours), 0),
            2
        ) AS capture_price
    FROM joined
    GROUP BY country, month
),
capture_floor0 AS (
    SELECT
        country,
		month, 	
        SUM(gen_mw * interval_hours * CASE WHEN price_raw < 0 THEN 0 ELSE price_raw END)
        / NULLIF(SUM(gen_mw * interval_hours), 0) AS capture_price_floor0_eur_per_mwh
    FROM joined
    GROUP BY country
),	
capture_price_annual AS (
    SELECT
        country,
        ROUND(
            SUM(weighted_price_sum) / NULLIF(SUM(weighted_gen_sum), 0),
            2
        ) AS annual_capture_price
    FROM capture_price
    GROUP BY country
),
capture_price_pivot AS (
    SELECT
        cp.country,
        COALESCE(MAX(CASE WHEN cp.month = 1 THEN cp.capture_price END), 0) AS Jan_cprice,
        COALESCE(MAX(CASE WHEN cp.month = 2 THEN cp.capture_price END), 0) AS Feb_cprice,
        COALESCE(MAX(CASE WHEN cp.month = 3 THEN cp.capture_price END), 0) AS Mar_cprice,
        COALESCE(MAX(CASE WHEN cp.month = 4 THEN cp.capture_price END), 0) AS Apr_cprice,
        COALESCE(MAX(CASE WHEN cp.month = 5 THEN cp.capture_price END), 0) AS May_cprice,
        COALESCE(MAX(CASE WHEN cp.month = 6 THEN cp.capture_price END), 0) AS Jun_cprice,
        COALESCE(MAX(CASE WHEN cp.month = 7 THEN cp.capture_price END), 0) AS Jul_cprice,
        COALESCE(MAX(CASE WHEN cp.month = 8 THEN cp.capture_price END), 0) AS Aug_cprice,
        COALESCE(MAX(CASE WHEN cp.month = 9 THEN cp.capture_price END), 0) AS Sep_cprice,
        COALESCE(MAX(CASE WHEN cp.month = 10 THEN cp.capture_price END), 0) AS Oct_cprice,
        COALESCE(MAX(CASE WHEN cp.month = 11 THEN cp.capture_price END), 0) AS Nov_cprice,
        COALESCE(MAX(CASE WHEN cp.month = 12 THEN cp.capture_price END), 0) AS Dec_cprice,
        ca.annual_capture_price
    FROM capture_price cp
    LEFT JOIN capture_price_annual ca ON cp.country = ca.country
    GROUP BY cp.country, ca.annual_capture_price
)
SELECT
    cp.country,
    ap.Jan_neg_hour, ap.Feb_neg_hour, ap.Mar_neg_hour, ap.Apr_neg_hour, ap.May_neg_hour, ap.Jun_neg_hour,
    ap.Jul_neg_hour, ap.Aug_neg_hour, ap.Sep_neg_hour, ap.Oct_neg_hour, ap.Nov_neg_hour, ap.Dec_neg_hour,
    ap.Annual_neg_hour,
    cp.Jan_cprice, cp.Feb_cprice, cp.Mar_cprice, cp.Apr_cprice, cp.May_cprice, cp.Jun_cprice,
    cp.Jul_cprice, cp.Aug_cprice, cp.Sep_cprice, cp.Oct_cprice, cp.Nov_cprice, cp.Dec_cprice,
    cp.annual_capture_price
FROM capture_price_pivot cp
LEFT JOIN agg_pivot ap
    ON cp.country = ap.Country
ORDER BY cp.country;

Error: Parser Error: syntax error at end of input

In [16]:
WITH raw AS (
    SELECT
        *,
        CAST(SUBSTR(filename, 6, 2) AS INTEGER) AS month,
        COUNT(*) FILTER (
            WHERE ResolutionCode = 'PT15M'
        ) OVER (
            PARTITION BY AreaDisplayName, "DateTime(UTC)"
        ) AS cnt_15m_same_ts
    FROM read_csv_auto(
        '2025_??_EnergyPrices_12.1.D_r3.csv', 
        filename=true
    )
    WHERE 
        ContractType = 'Day-ahead'
        AND Sequence NOT IN ('2', '3')
),
agg AS (
    SELECT
        AreaDisplayName AS Country,
        month,
        SUM(
            CASE
                WHEN ResolutionCode = 'PT15M'
                     AND "Price[Currency/MWh]" < 0
                THEN 0.25
                ELSE 0
            END
        ) AS hours_15m,
        SUM(
            CASE
                WHEN ResolutionCode = 'PT60M'
                     AND cnt_15m_same_ts = 0
                     AND "Price[Currency/MWh]" < 0
                THEN 1
                ELSE 0
            END
        ) AS hours_60m
    FROM raw
    GROUP BY AreaDisplayName, month
),
agg_pivot AS (
    SELECT
        Country,
        COALESCE(SUM(CASE WHEN month = 1 THEN hours_15m + hours_60m END), 0) AS Jan_neg_hour,
        COALESCE(SUM(CASE WHEN month = 2 THEN hours_15m + hours_60m END), 0) AS Feb_neg_hour,
        COALESCE(SUM(CASE WHEN month = 3 THEN hours_15m + hours_60m END), 0) AS Mar_neg_hour,
        COALESCE(SUM(CASE WHEN month = 4 THEN hours_15m + hours_60m END), 0) AS Apr_neg_hour,
        COALESCE(SUM(CASE WHEN month = 5 THEN hours_15m + hours_60m END), 0) AS May_neg_hour,
        COALESCE(SUM(CASE WHEN month = 6 THEN hours_15m + hours_60m END), 0) AS Jun_neg_hour,
        COALESCE(SUM(CASE WHEN month = 7 THEN hours_15m + hours_60m END), 0) AS Jul_neg_hour,
        COALESCE(SUM(CASE WHEN month = 8 THEN hours_15m + hours_60m END), 0) AS Aug_neg_hour,
        COALESCE(SUM(CASE WHEN month = 9 THEN hours_15m + hours_60m END), 0) AS Sep_neg_hour,
        COALESCE(SUM(CASE WHEN month = 10 THEN hours_15m + hours_60m END), 0) AS Oct_neg_hour,
        COALESCE(SUM(CASE WHEN month = 11 THEN hours_15m + hours_60m END), 0) AS Nov_neg_hour,
        COALESCE(SUM(CASE WHEN month = 12 THEN hours_15m + hours_60m END), 0) AS Dec_neg_hour,
        COALESCE(SUM(hours_15m + hours_60m), 0) AS Annual_neg_hour
    FROM agg
    GROUP BY Country
),
prices_raw AS (
    SELECT
        ep.*,
        CAST(SUBSTR(ep.filename, 6, 2) AS INTEGER) AS month,
        COUNT(*) FILTER (
            WHERE ResolutionCode = 'PT60M'
        ) OVER (
            PARTITION BY AreaDisplayName, "DateTime(UTC)"
        ) AS cnt_60m_same_ts
    FROM read_csv_auto('2025_??_EnergyPrices_12.1.D_r3.csv', filename=true) AS ep
    WHERE
        ep.ContractType = 'Day-ahead'
        AND ep.Sequence NOT IN ('2', '3')
),
prices_dedup AS (
    SELECT *
    FROM prices_raw
    WHERE
        ResolutionCode = 'PT60M'
        OR (ResolutionCode = 'PT15M' AND cnt_60m_same_ts = 0)
),
joined AS (
    SELECT
        ep.AreaDisplayName AS country,
        ep."DateTime(UTC)",
        ep.ResolutionCode,
        ep.month,
        CASE
            WHEN ep."Price[Currency/MWh]" < 0 THEN 0
            ELSE ep."Price[Currency/MWh]"
        END AS price_nonneg,
        ep."Price[Currency/MWh]" AS price_raw,
        ag."ActualGenerationOutput" AS gen_mw,
        CASE
            WHEN ep.ResolutionCode = 'PT15M' THEN 0.25
            WHEN ep.ResolutionCode = 'PT60M' THEN 1.0
            ELSE 1.0
        END AS interval_hours
    FROM prices_dedup AS ep
    JOIN read_csv_auto(
            '2025_??_AggregatedGenerationPerType_16.1.B_C.csv',
            strict_mode=false,
            ignore_errors=true,
            filename=true
        ) AS ag
      ON ep.AreaCode = ag.AreaCode
     AND ep."DateTime(UTC)" = ag."DateTime"
     AND ep.month = CAST(SUBSTR(ag.filename, 6, 2) AS INTEGER)
    WHERE
        ag.ProductionType = 'Solar'
        AND ag."ActualGenerationOutput" > 0
),
-- capture price calculation 
capture_price AS (
    SELECT
        country,
        month,
        SUM(gen_mw * interval_hours * price_raw) AS weighted_price_sum,
        SUM(gen_mw * interval_hours) AS weighted_gen_sum,
        ROUND(
            SUM(gen_mw * interval_hours * price_raw)
            / NULLIF(SUM(gen_mw * interval_hours), 0),
            2
        ) AS capture_price
    FROM joined
    GROUP BY country, month
),
-- floor-priced capture price per month
capture_floor0 AS (
    SELECT
        country,
        month,
        ROUND(SUM(gen_mw * interval_hours * CASE WHEN price_raw < 0 THEN 0 ELSE price_raw END)
        / NULLIF(SUM(gen_mw * interval_hours), 0),2) AS capture_price_floor0
    FROM joined
    GROUP BY country, month
),
-- annual capture price 
capture_price_annual AS (
    SELECT
        country,
        ROUND(
            SUM(weighted_price_sum) / NULLIF(SUM(weighted_gen_sum), 0),
            2
        ) AS annual_capture_price
    FROM capture_price
    GROUP BY country
),
-- capture price pivoting 
capture_price_pivot AS (
    SELECT
        cp.country,
        COALESCE(MAX(CASE WHEN cp.month = 1 THEN cp.capture_price END), 0) AS Jan_cprice,
        COALESCE(MAX(CASE WHEN cp.month = 2 THEN cp.capture_price END), 0) AS Feb_cprice,
        COALESCE(MAX(CASE WHEN cp.month = 3 THEN cp.capture_price END), 0) AS Mar_cprice,
        COALESCE(MAX(CASE WHEN cp.month = 4 THEN cp.capture_price END), 0) AS Apr_cprice,
        COALESCE(MAX(CASE WHEN cp.month = 5 THEN cp.capture_price END), 0) AS May_cprice,
        COALESCE(MAX(CASE WHEN cp.month = 6 THEN cp.capture_price END), 0) AS Jun_cprice,
        COALESCE(MAX(CASE WHEN cp.month = 7 THEN cp.capture_price END), 0) AS Jul_cprice,
        COALESCE(MAX(CASE WHEN cp.month = 8 THEN cp.capture_price END), 0) AS Aug_cprice,
        COALESCE(MAX(CASE WHEN cp.month = 9 THEN cp.capture_price END), 0) AS Sep_cprice,
        COALESCE(MAX(CASE WHEN cp.month = 10 THEN cp.capture_price END), 0) AS Oct_cprice,
        COALESCE(MAX(CASE WHEN cp.month = 11 THEN cp.capture_price END), 0) AS Nov_cprice,
        COALESCE(MAX(CASE WHEN cp.month = 12 THEN cp.capture_price END), 0) AS Dec_cprice,
        ca.annual_capture_price
    FROM capture_price cp
    LEFT JOIN capture_price_annual ca ON cp.country = ca.country
    GROUP BY cp.country, ca.annual_capture_price
),
-- pivot for floor-priced capture prices per month
capture_floor0_pivot AS (
    SELECT
        cf.country,
        COALESCE(MAX(CASE WHEN cf.month = 1 THEN cf.capture_price_floor0 END), 0) AS Jan_cprice_floor0,
        COALESCE(MAX(CASE WHEN cf.month = 2 THEN cf.capture_price_floor0 END), 0) AS Feb_cprice_floor0,
        COALESCE(MAX(CASE WHEN cf.month = 3 THEN cf.capture_price_floor0 END), 0) AS Mar_cprice_floor0,
        COALESCE(MAX(CASE WHEN cf.month = 4 THEN cf.capture_price_floor0 END), 0) AS Apr_cprice_floor0,
        COALESCE(MAX(CASE WHEN cf.month = 5 THEN cf.capture_price_floor0 END), 0) AS May_cprice_floor0,
        COALESCE(MAX(CASE WHEN cf.month = 6 THEN cf.capture_price_floor0 END), 0) AS Jun_cprice_floor0,
        COALESCE(MAX(CASE WHEN cf.month = 7 THEN cf.capture_price_floor0 END), 0) AS Jul_cprice_floor0,
        COALESCE(MAX(CASE WHEN cf.month = 8 THEN cf.capture_price_floor0 END), 0) AS Aug_cprice_floor0,
        COALESCE(MAX(CASE WHEN cf.month = 9 THEN cf.capture_price_floor0 END), 0) AS Sep_cprice_floor0,
        COALESCE(MAX(CASE WHEN cf.month = 10 THEN cf.capture_price_floor0 END), 0) AS Oct_cprice_floor0,
        COALESCE(MAX(CASE WHEN cf.month = 11 THEN cf.capture_price_floor0 END), 0) AS Nov_cprice_floor0,
        COALESCE(MAX(CASE WHEN cf.month = 12 THEN cf.capture_price_floor0 END), 0) AS Dec_cprice_floor0
    FROM capture_floor0 cf
    GROUP BY cf.country
)

SELECT
    cp.country,
    -- negative hours - per month
    ap.Jan_neg_hour, ap.Feb_neg_hour, ap.Mar_neg_hour, ap.Apr_neg_hour, ap.May_neg_hour, ap.Jun_neg_hour,
    ap.Jul_neg_hour, ap.Aug_neg_hour, ap.Sep_neg_hour, ap.Oct_neg_hour, ap.Nov_neg_hour, ap.Dec_neg_hour,
    ap.Annual_neg_hour,
    -- capture price per month 
    cp.Jan_cprice, cp.Feb_cprice, cp.Mar_cprice, cp.Apr_cprice, cp.May_cprice, cp.Jun_cprice,
    cp.Jul_cprice, cp.Aug_cprice, cp.Sep_cprice, cp.Oct_cprice, cp.Nov_cprice, cp.Dec_cprice,
    cp.annual_capture_price,
    -- floor-priced capture price per month
    cfp.Jan_cprice_floor0, cfp.Feb_cprice_floor0, cfp.Mar_cprice_floor0, cfp.Apr_cprice_floor0,
    cfp.May_cprice_floor0, cfp.Jun_cprice_floor0, cfp.Jul_cprice_floor0, cfp.Aug_cprice_floor0,
    cfp.Sep_cprice_floor0, cfp.Oct_cprice_floor0, cfp.Nov_cprice_floor0, cfp.Dec_cprice_floor0
FROM capture_price_pivot cp
LEFT JOIN agg_pivot ap
    ON cp.country = ap.Country
LEFT JOIN capture_floor0_pivot cfp
    ON cp.country = cfp.country
ORDER BY cp.country;


,country,Jan_neg_hour,Feb_neg_hour,Mar_neg_hour,Apr_neg_hour,May_neg_hour,Jun_neg_hour,Jul_neg_hour,Aug_neg_hour,Sep_neg_hour,Oct_neg_hour,Nov_neg_hour,Dec_neg_hour,Annual_neg_hour,Jan_cprice,Feb_cprice,Mar_cprice,Apr_cprice,May_cprice,Jun_cprice,Jul_cprice,Aug_cprice,Sep_cprice,Oct_cprice,Nov_cprice,Dec_cprice,annual_capture_price,Jan_cprice_floor0,Feb_cprice_floor0,Mar_cprice_floor0,Apr_cprice_floor0,May_cprice_floor0,Jun_cprice_floor0,Jul_cprice_floor0,Aug_cprice_floor0,Sep_cprice_floor0,Oct_cprice_floor0,Nov_cprice_floor0,Dec_cprice_floor0
0,Austria (AT),0.0,0.0,7.0,76.0,92.0,115.0,6.0,59.0,23.0,0.00,0.00,0.0,378.00,130.18,128.95,61.74,28.01,4.91,14.35,58.58,31.95,48.57,89.77,0.0,0.0,46.83,130.18,128.95,61.75,37.64,23.26,18.68,58.65,34.19,50.30,89.77,0.0,0.0
1,Belgium (BE),0.0,0.0,32.0,80.0,122.0,127.0,11.0,58.0,58.0,32.25,0.00,0.0,520.25,117.01,125.60,51.84,24.38,4.18,21.23,57.46,34.05,36.34,66.43,0.0,0.0,39.72,117.01,125.60,53.82,33.04,25.77,24.48,57.55,36.14,38.16,66.59,0.0,0.0
2,Bulgaria (BG),0.0,0.0,6.0,46.0,35.0,28.0,1.0,32.0,22.0,0.00,0.00,0.0,170.00,124.80,136.81,56.14,54.88,52.98,45.36,73.79,40.79,59.84,116.71,0.0,0.0,64.88,124.80,136.81,56.14,56.84,54.37,45.55,73.79,41.28,60.24,116.71,0.0,0.0
3,Croatia (HR),0.0,0.0,9.0,68.0,61.0,48.0,5.0,43.0,23.0,0.25,0.00,0.0,257.25,126.80,135.68,64.34,52.57,47.00,40.71,78.41,40.93,65.49,95.65,0.0,0.0,65.74,126.80,135.68,64.35,55.44,52.09,41.42,78.41,41.63,66.07,95.65,0.0,0.0
4,Czech Republic (CZ),0.0,0.0,10.0,72.0,81.0,69.0,6.0,54.0,23.0,8.00,0.25,0.0,323.25,125.65,119.16,57.42,34.94,22.86,29.31,66.13,38.91,56.72,82.73,0.0,0.0,52.43,125.65,119.16,57.44,41.21,35.27,31.37,66.15,40.27,58.29,82.73,0.0,0.0
5,DE-LU,14.0,0.0,30.0,75.0,129.0,141.0,12.0,64.0,60.0,49.75,0.00,0.0,574.75,116.50,116.18,53.64,30.09,18.83,20.14,59.84,39.42,44.52,76.91,0.0,0.0,45.76,116.51,116.18,53.78,36.77,32.14,23.91,59.89,40.84,46.48,77.04,0.0,0.0
6,DK1,0.0,0.0,29.0,74.0,107.0,116.0,12.0,46.0,27.0,30.00,0.00,0.0,441.00,115.99,110.53,57.55,40.71,39.08,29.67,60.98,46.94,48.21,75.35,0.0,0.0,52.07,115.99,110.53,57.61,41.84,39.86,30.34,60.99,47.45,48.62,75.41,0.0,0.0
7,DK2,0.0,0.0,17.0,58.0,69.0,66.0,4.0,31.0,5.0,15.00,0.00,0.0,265.00,118.54,113.28,52.23,40.01,36.28,23.10,59.29,42.29,51.90,76.55,0.0,0.0,49.58,118.54,113.28,52.26,40.85,36.85,23.81,59.30,42.67,51.91,76.55,0.0,0.0
8,Estonia (EE),0.0,0.0,9.0,35.0,23.0,70.0,18.0,20.0,7.0,11.25,7.25,0.0,200.50,108.66,146.32,53.64,30.33,39.08,31.54,26.69,54.84,64.39,76.86,0.0,0.0,44.26,108.66,146.32,53.64,30.51,39.15,32.11,26.69,54.88,64.40,76.86,0.0,0.0
9,Finland (FI),0.0,3.0,41.0,69.0,100.0,126.0,18.0,37.0,19.0,21.75,10.00,0.0,444.75,67.65,81.17,55.15,28.12,23.44,27.59,25.96,54.26,47.08,57.51,0.0,0.0,37.46,67.65,81.17,55.16,28.32,23.52,28.18,25.96,54.30,47.08,57.51,0.0,0.0
